# Naive Bayes

- 思想：对于给定的待分类项x，通过学习到的模型计算后验概率分布，即：在此项出现的条件下各个目标类别出现的概率，将后验概率最大的类作为x所属的类别。后验概率根据贝叶斯定理计算。 

- $P(类别|特征)=\frac{P(特征|类别)*P(类别)}{P(特征)}$

- 而朴素贝叶斯推断，是在贝叶斯推断的基础上，对条件概率分布做了条件独立性的假设。因此可得朴素贝叶斯分类器的表达式。因为以自变量之间的独立（条件特征独立）性和连续变量的正态性假设为前提，就会导致算法精度在某种程度上受影响

公式: $\hat y=arg max_{c \in Y} P(c)\prod_{i=1}^{d}P(x_{i}|c)$

#  贝叶斯分类器和贝叶斯学习有什么不同？


- 前者：通过最大后验概率进行单点估计；

- 后者：进行分布估计。

朴素贝叶斯算法可以进行多分类，因此使用的还是原版手写识别数据集；但是在实现过程中发现，对于朴素贝叶斯算法来说：数据维度或取值范围过大（mnist每行数据有784维，每维数据取值0～255 共256个值），都会引起概率计算过小甚至为零的问题，尤其是在做了拉普拉斯平滑的步骤之后。

因此在使用过程中，对图像数据进行二值化操作，把数据取值范围从256维压缩到2维，可以在一定程度上降低概率弥散的情况，但同时也损失了一定的数据信息。这部分操作直接在代码中完成，就不提前做生成新的数据集了。


优点：

（1） 算法逻辑简单,易于实现

（2）分类过程中时空开销小

缺点：

理论上，朴素贝叶斯模型与其他分类方法相比具有最小的误差率。但是实际上并非总是如此，这是因为朴素贝叶斯模型假设属性之间相互独立，这个假设在实际应用中往往是不成立的，在属性个数比较多或者属性之间相关性较大时，分类效果不好。

而在属性相关性较小时，朴素贝叶斯性能最为良好。对于这一点，有半朴素贝叶斯之类的算法通过考虑部分关联性适度改进。

朴素贝叶斯算法之所以朴素，是因为它对条件概率分布作了条件独立的假设；
通过基于贝叶斯定理和特征条件独立假设，对给定的训练数据集，首先基于特征条件独立假设学习输入输出的联合概率分布；然后基于学习的模型，对给定的输入 x ，利用贝叶斯定理求出后验概率最大的输出 y 
基于极大斯然估计的朴素贝叶斯算法步骤如下：


In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Binarizer
#定义计算预测精度的函数
def calc_accuracy(y_pred, y_truth):
    assert len(y_pred) == len(y_truth)
    n = len(y_pred)

    hit_count = 0
    for i in range(0, n):
        if y_pred[i] == y_truth[i]:
            hit_count += 1

    print("Predicting accuracy %f\n" % (hit_count / n))


class NaiveBayes(object):
    def __init__(self, _lambda, Sj, K):
        self._lambda = _lambda
        self.Sj = Sj  # Feature Dimension (Simple assume to the same)
        self.K = K  # Label range

    # Use bayes estimate
    # Not max-likelihood estimate, avoid probability is 0
    def train(self, X_train, y_train):
        # Calculate prior probability & conditional probability
        N = len(y_train)
        D = X_train.shape[1]  # Dimension

        prior = np.full(self.K, 0)
        condition = np.full((self.K, D, self.Sj), 0)
        # conditional_probability = np.full((self.K, D, self.Sj), 0.)

        for i in range(0, N):
            prior[y_train[i]] += 1
            for j in range(0, D):
                condition[y_train[i]][j][X_train[i][j]] += 1

        prior_probability = (prior + self._lambda) / (N + self.K*self._lambda)

        # Too Slow
        # for i in range(0, self.K):
        #     for j in range(0, D):
        #         for k in range(0, self.Sj):
        #             conditional_probability[i][j][k] = \
        #                 (condition[i][j][k] + self._lambda) / (sum(condition[i][j]) + self.Sj*self._lambda)

        return prior_probability, condition  # , conditional_probability

    def predict(self, prior_probability, condition, X_test):
        n = len(X_test)
        d = X_test.shape[1]

        predict_label = np.full(n, -1)

        for i in range(0, n):
            predict_probability = np.full(self.K, 1.)
            to_predict = X_test[i]

            for j in range(0, self.K):
                prior_prob = prior_probability[j]

                # If d or self.Sj is large, predict_probability gets close to 0
                for k in range(0, d):
                    conditional_probability = \
                        (condition[j][k][to_predict[k]] + self._lambda) / (sum(condition[j][k]) + self.Sj*self._lambda)
                    predict_probability[j] *= conditional_probability

                predict_probability[j] *= prior_prob

            predict_label[i] = np.argmax(predict_probability)

            print("Sample %d predicted as %d" % (i, predict_label[i]))

        return predict_label


def example_large():
    mnist_data = pd.read_csv("../data/mnist.csv")
    mnist_values = mnist_data.values

    images = mnist_values[::, 1::]
    labels = mnist_values[::, 0]

    X_train, X_test, y_train, y_test = train_test_split(
        images, labels, test_size=100, random_state=42
    )

    # Binary the image to avoid predict_probability gets 0
    binarizer_train = Binarizer(threshold=127).fit(X_train)
    X_train_binary = binarizer_train.transform(X_train)

    binarizer_test = Binarizer(threshold=127).fit(X_test)
    X_test_binary = binarizer_test.transform(X_test)

    # Laplace Smoothing
    # X values 0~255 = 256 Every axis has the same range
    # Y values 0~9 = 10
    naive_bayes = NaiveBayes(_lambda=1, Sj=2, K=10)

    print("Start naive bayes training...")
    prior, conditional = naive_bayes.train(X_train=X_train_binary, y_train=y_train)

    print("Testing on %d samples..." % len(X_test))
    y_predicted = naive_bayes.predict(prior_probability=prior,
                                      condition=conditional,
                                      X_test=X_test_binary)

    calc_accuracy(y_pred=y_predicted, y_truth=y_test)


def example_small():
    X_train = np.asarray([[0, 0], [0, 1], [0, 1], [0, 0], [0, 0],
                          [1, 0], [1, 1], [1, 1], [1, 2], [1, 2],
                          [2, 2], [2, 1], [2, 1], [2, 2], [2, 2]])

    y_train = np.asarray([0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0])

    X_test = np.asarray([[1, 0]])

    naive_bayes = NaiveBayes(_lambda=1, Sj=3, K=2)

    print("Start naive bayes training...")
    prior, conditional = naive_bayes.train(X_train=X_train, y_train=y_train)

    print("Testing on %d samples..." % len(X_test))
    naive_bayes.predict(prior_probability=prior,
                        condition=conditional,
                        X_test=X_test)


if __name__ == "__main__":
    # example_small()
    example_large()
